In [3]:
%matplotlib inline
import vtk
import dicom
import os
import numpy as np
from matplotlib import pyplot as plt, cm
from IPython.display import Image
from skxray.io.tiff_io import tiff_io


ImportError: cannot import name tiff_io

In [2]:
pwd

u'/home/giltis/dev/my_src/Demos/demos/tomography/visualization'

In [7]:
img_path = '../data/uCT/'
fname_abv = 'NSLS_shale_smpl-2_AbvFe.tiff'
fname_blw = 'NSLS_shale_smpl-2_BlwFe.tiff'


In [9]:
reader = vtk.vtkTIFFReader()
reader.SetFileName(img_path+fname_abv)
#reader = vtk.vtkDICOMImageReader()                                 
#reader.SetDirectoryName(img_path)
reader.Update()

In [10]:
reader.GetDataExtent()

(0, 0, 0, 0, 0, 0)

In [5]:
_extent = reader.GetDataExtent()
print _extent
pixel_dims_vtk = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]
pixel_spacing_vtk = reader.GetPixelSpacing()
print pixel_dims_vtk
print pixel_spacing_vtk

(0, 511, 0, 511, 0, 244)
[512, 512, 245]
(1.0, 1.0, 1.0)


In [6]:
threshold = vtk.vtkImageThreshold()
threshold.SetInputConnection(reader.GetOutputPort())
threshold.ThresholdByLower(450)
threshold.ReplaceInOn()
threshold.SetInValue(0)
threshold.ReplaceOutOn()
threshold.SetOutValue(1)
threshold.Update()

In [7]:
dmc = vtk.vtkDiscreteMarchingCubes()
dmc.SetInputConnection(threshold.GetOutputPort())
dmc.GenerateValues(1,1,1)
dmc.Update()

In [8]:
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(dmc.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(0.5,0.5,0.5)
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
interactiveRender = vtk.vtkRenderWindowInteractor()
interactiveRender.SetRenderWindow(renWin)

camera = renderer.MakeCamera()
camera.SetPosition(-500.0, 245.5, 122.0)
camera.SetFocalPoint(301.0, 245.5, 122.0)
camera.SetViewAngle(30.0)
camera.SetRoll(-90.0)
renderer.SetActiveCamera(camera)
renWin.SetSize(600,600)

interactiveRender.Initialize()
renWin.Render()
interactiveRender.Start()